In [6]:
# Импортируем PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, round

# Создаем сессию Spark
spark = SparkSession.builder \
    .appName("VideoGameSales") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

# 1. Прочитать датасет
df = spark.read.csv("file:///home/jovyan/kt8/vgsales.csv", header=True, inferSchema=True)

# Проверим данные
df.show(5)
df.printSchema()

# 2. Нормализовать данные (например, привести строки к нижнему регистру и убрать пробелы)
from pyspark.sql.functions import lower, trim

columns_to_normalize = ['Name', 'Platform', 'Genre', 'Publisher']
for c in columns_to_normalize:
    df = df.withColumn(c, lower(trim(col(c))))

# 3. Создать столбец с общими продажами игр по годам
df_year_sales = df.groupBy("Year").agg(
    round(spark_sum("Global_Sales"),2).alias("Total_Global_Sales_Per_Year")
)

# Присоединим к исходному датасету
df = df.join(df_year_sales, on="Year", how="left")

# 4. Создать столбец с общими продажами игр по годам, платформе и региону
regions = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]
for region in regions:
    df_region_sales = df.groupBy("Year", "Platform").agg(
        round(spark_sum(col(region)),2).alias(f"Total_{region}_Per_Year_Platform")
    )
    df = df.join(df_region_sales, on=["Year", "Platform"], how="left")

# 5. Создать столбец с долей продаж игр по годам, платформе и региону
for region in regions:
    df = df.withColumn(f"{region}_Share", round(col(region) / col(f"Total_{region}_Per_Year_Platform"), 4))

# 6. Создать столбец с долей продаж игр по издателю
df_publisher_sales = df.groupBy("Publisher").agg(
    round(spark_sum("Global_Sales"),2).alias("Total_Global_Sales_Per_Publisher")
)
total_global_sales = df.agg(spark_sum("Global_Sales").alias("Global_Sales_Total")).collect()[0]["Global_Sales_Total"]
df = df.join(df_publisher_sales, on="Publisher", how="left") \
       .withColumn("Publisher_Share", round(col("Total_Global_Sales_Per_Publisher") / total_global_sales, 4))

# 7. Создать столбец с суммой продаж каждой игры
df = df.withColumn("Total_Sales", sum([col(r) for r in regions]) + col("Global_Sales"))

# 8. Загрузить датасет в parquet
df.write.mode("overwrite").parquet("hdfs://namenode:9000/jupyter/kt8/vgs.parquet")

# Проверим результат
df.show(5)


+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|Rank|                Name|Platform|Year|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
+----+--------------------+--------+----+------------+---------+--------+--------+--------+-----------+------------+
|   1|          Wii Sports|     Wii|2006|      Sports| Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
|   2|   Super Mario Bros.|     NES|1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|
|   3|      Mario Kart Wii|     Wii|2008|      Racing| Nintendo|   15.85|   12.88|    3.79|       3.31|       35.82|
|   4|   Wii Sports Resort|     Wii|2009|      Sports| Nintendo|   15.75|   11.01|    3.28|       2.96|        33.0|
|   5|Pokemon Red/Pokem...|      GB|1996|Role-Playing| Nintendo|   11.27|    8.89|   10.22|        1.0|       31.37|
+----+--------------------+--------+----+------------+---------+